<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries help us fetch financial data, manipulate it, and create visualizations

In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import WeekOfMonth
import yfinance as yf
from matplotlib import pyplot as plt

### Fetch option data for our stock

We'll gather option data for a specific stock ticker and organize it into a dataframe.

In [ ]:
ticker = "SPY"
stock = yf.Ticker(ticker)
expirations = stock.options

In [ ]:
all_options = []
for dt in expirations:
    options = stock.option_chain(dt)
    calls = options.calls
    calls['expirationDate'] = dt
    all_options.append(calls)

In [ ]:
options_df = pd.concat(all_options)
options_df['daysToExpiration'] = (pd.to_datetime(options_df['expirationDate']) - pd.Timestamp.now()).dt.days

We start by defining our stock ticker as "SPY" and use yfinance to fetch its data. We then loop through all available expiration dates, collecting call option data for each. We combine this data into a single dataframe and calculate the days to expiration for each option.

### Filter and clean our option data

Now we'll filter our options to a specific strike price and clean up the data.

In [ ]:
strike = 590
filtered_options = options_df[options_df['strike'] == strike]

In [ ]:
def clean(df):
    cols = ["daysToExpiration", "impliedVolatility"]
    return (
        df[cols]
        .dropna(subset=["impliedVolatility"])
        .drop_duplicates("daysToExpiration")
        .sort_values("daysToExpiration")
        .set_index("daysToExpiration")
    )

In [ ]:
def build_iv_grid(df):
    df_iv = clean(df)
    df_iv = df_iv.where(df_iv >= 0.01, np.nan)
    df_iv = df_iv[df_iv.index > 30]
    expiry_grid = np.arange(df_iv.index.min(), df_iv.index.max() + 1, 1.0)
    return (
        df_iv.reindex(expiry_grid)
        .interpolate(method="linear")
        .rename_axis("daysToExpiration")
    ).dropna()

In [ ]:
call_structure = build_iv_grid(options_df)

We filter our options to focus on a specific strike price. The clean function keeps only the days to expiration and implied volatility columns, removing any duplicates or missing data. The build_iv_grid function further processes this data, creating a grid of implied volatilities across different expiration dates.

### Visualize the implied volatility term structure

Finally, we'll create a plot to visualize the implied volatility term structure.

In [ ]:
plt.plot(call_structure.index, call_structure.impliedVolatility)
plt.xlabel('Days to Expiration')
plt.ylabel('Implied Volatility')
plt.title(f'Implied Volatility Term Structure for {ticker} (Strike: {strike})')
plt.grid(True)
plt.show()

We use matplotlib to create a line plot of our implied volatility data. The x-axis shows the days to expiration, while the y-axis shows the implied volatility. We add labels to our axes and a title that includes our stock ticker and strike price. The grid helps make the chart easier to read.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.